# Baseline Yolo8n wandb

In [1]:
!pip install -U ultralytics wandb
!pip install dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 35.3 MB/s eta 0:00:00
  Attempting uninstall: ultralytics
    Found existing installation: ultralytics 8.3.123
    Uninstalling ultralytics-8.3.123:
      Successfully uninstalled ultralytics-8.3.123


In this Notebook the Baseline Model will be trained with sweeps  
Please preregister on the homepage and provide the necessary API Key in the .env file  
Maybee using Sweeps could ease the process

In [2]:
# Enable W&B logging for Ultralytics
!yolo settings wandb=True

JSONDict("/home/jovyan/.config/Ultralytics/settings.json"):
{
  "settings_version": "0.0.6",
  "datasets_dir": "/home/jovyan/DSPRO2/M-AI-ZE-Maize-diseases-detection/notebooks/datasets",
  "weights_dir": "weights",
  "runs_dir": "runs",
  "uuid": "18e178169b4eaa957c2b01da8ec0b439a0c38ab85c8749c9c8f8f83149ac02ba",
  "sync": true,
  "api_key": "",
  "openai_api_key": "",
  "clearml": true,
  "comet": true,
  "dvc": true,
  "hub": true,
  "mlflow": true,
  "neptune": true,
  "raytune": true,
  "tensorboard": false,
  "wandb": true,
  "vscode_msg": true
}
💡 Learn more about Ultralytics Settings at https://docs.ultralytics.com/quickstart/#ultralytics-settings


In [3]:
# check the .env file 
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()

# Get and print the WANDB_API_KEY
wandb_api_key = os.getenv("WANDB_API_KEY")
print(f"WANDB_API_KEY: [{wandb_api_key[:4]}...]")

WANDB_API_KEY: [69ca...]


In [4]:
import wandb
wandb.login(key=wandb_api_key)

wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/jovyan/.netrc
wandb: Currently logged in as: rueedi-tobias (rueedi-tobias-hochschule-luzern) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

## Sweep Training for Baseline


In [5]:
dataset_yaml = "/exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID01/data.yaml"

In [10]:
sweep_config = {
    "method": "random",
    "name": "yolov11_comparison_sweep",
    "metric": {
        "name": "metrics/mAP50",
        "goal": "maximize"
    },
    "parameters": {
        "model": {
            "values": [
                "yolo11n.pt",
                "yolo11s.pt",
                "yolo11m.pt",
                "yolo11l.pt",
                "yolo11x.pt"
            ]
        },
        "epochs": {"values": [10, 20]},
        "imgsz": {"values": [640]},
        "batch": {"values": [8, 16]},
        "lr0": {"values": [0.001, 0.005, 0.01]},
        "momentum": {"values": [0.85, 0.937]}
    }
}


In [11]:
import wandb
from ultralytics import YOLO

def train_yolo_sweep():
    run = wandb.init()
    config = run.config

    
    model = YOLO(config.model)

    model.train(
        data=dataset_yaml,
        epochs=config.epochs,
        imgsz=config.imgsz,
        lr0=config.lr0,
        batch=config.batch,
        name=f"sweep_{run.id}",
        project="maize_disease_detection",
        exist_ok=True
    )
    run.finish()


In [ ]:
sweep_id = wandb.sweep(sweep=sweep_config, project="maize_disease_detection")
wandb.agent(sweep_id, function=train_yolo_sweep, count=30)

Create sweep with ID: 6cvyz1l7
Sweep URL: https://wandb.ai/rueedi-tobias-hochschule-luzern/maize_disease_detection/sweeps/6cvyz1l7


wandb: Agent Starting Run: 1mst8p7z with config:
wandb: 	batch: 16
wandb: 	epochs: 20
wandb: 	imgsz: 640
wandb: 	lr0: 0.001
wandb: 	model: yolo11n.pt
wandb: 	momentum: 0.937
wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Ultralytics 8.3.124 🚀 Python-3.12.10 torch-2.7.0+cu126 CUDA:0 (NVIDIA A16, 14891MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID01/data.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=maize_disease_detection, name=sweep_1mst8p7z, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=F

train: Scanning /exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID01/labels/train.cache... 10858 images, 0 backgrounds, 0 corrupt: 100%|██████████| 10858/10858 [00:00<?, ?it/s]


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1517.2±663.6 MB/s, size: 56.1 KB)


...

       5/10      5.04G      1.939      1.675      1.451         84        640: 100%|██████████| 679/679 [04:27<00:00,  2.54it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 43/43 [00:15<00:00,  2.76it/s]


                   all       1357      10668      0.551      0.458      0.469      0.202

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       6/10      5.04G      1.909      1.625      1.434         67        640: 100%|██████████| 679/679 [04:27<00:00,  2.53it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 43/43 [00:16<00:00,  2.59it/s]


                   all       1357      10668      0.563      0.463      0.483      0.226

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       7/10      5.04G      1.889      1.584      1.414         88        640:  19%|█▉        | 131/679 [00:51<03:34,  2.56it/s]